In [2]:
!pip install ftfy regex tqdm 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.3 MB/s 


In [3]:
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-0w4vilgz
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-0w4vilgz
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369387 sha256=339199e0aa983c12faa4fb2dd9f6363c18d53736a4b957944a2f13bafbe4dfc4
  Stored in directory: /tmp/pip-ephem-wheel-cache-o40tyo78/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [4]:
import torch
import clip 
from PIL import Image 

device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)

cuda


In [14]:
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:18<00:00, 18.7MiB/s]


In [5]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 63.5 MB/s 
     |████████████████████████████████| 101 kB 13.6 MB/s 
     |████████████████████████████████| 596 kB 82.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 4.2 MB/s 
     |████████████████████████████████| 212 kB 85.6 MB/s 
     |████████████████████████████████| 1.1 MB 85.0 MB/s 
     |████████████████████████████████| 140 kB 95.3 MB/s 
     |████████████████████████████████| 127 kB 87.8 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 271 kB 89.8 MB/s 
     |█████████████

In [6]:
from datasets import load_dataset

dataset = load_dataset("mnist")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset mnist downloaded and prepared to /root/.cache/huggingface/datasets/mnist/mnist/1.0.0/fda16c03c4ecfb13f165ba7e29cf38129ce035011519968cdaf74894ce91c9d4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [8]:
mnist = dataset['test']

In [10]:
mnist[1]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x7FF14F4FBC50>,
 'label': 2}

In [11]:
mnist[1]['image']

In [12]:
mnist[1]['label']

2

In [60]:
image = preprocess(mnist[1]['image']).unsqueeze(0).to(device)

label_list = ["number 0","number 1","number 2","number 3","number 4","number 5","number 6","number 7","number 8","number 9"]

label_list = ["0","1","2","3","4","5","6","7","8","9"]

label_list = []

for i in range(10):
  strr = f"A handwritten number {i}"
  label_list.append(strr)

print(label_list)
text = clip.tokenize(label_list).to(device)

['A handwritten number 0', 'A handwritten number 1', 'A handwritten number 2', 'A handwritten number 3', 'A handwritten number 4', 'A handwritten number 5', 'A handwritten number 6', 'A handwritten number 7', 'A handwritten number 8', 'A handwritten number 9']


In [61]:
with torch.no_grad():
  image_features = model.encode_image(image)
  text_features = model.encode_text(text)

  logits_per_image, logits_per_text = model(image, text)
  probs = logits_per_image.softmax(dim=-1).cpu().numpy()

  print("Label probs:",probs)
  #print("Predict Label:" ,probs.index(max(probs[0]))

Label probs: [[0.1628  0.04523 0.02576 0.06476 0.07336 0.08716 0.07574 0.2644  0.0957
  0.1051 ]]


In [62]:
import numpy as np 

probs

print(np.argmax(probs[0]))

7


## MNIST로 Zero-shot Image Classification 직접 실험 

In [68]:
mnist = dataset['test']

print(mnist)


Dataset({
    features: ['image', 'label'],
    num_rows: 10000
})


AttributeError: ignored

In [70]:
model, preprocess = clip.load("ViT-B/32", device=device)


In [71]:
label_list = []

for i in range(10):
  strr = f"A handwritten number {i}"
  label_list.append(strr)

print(label_list)
text = clip.tokenize(label_list).to(device)

['A handwritten number 0', 'A handwritten number 1', 'A handwritten number 2', 'A handwritten number 3', 'A handwritten number 4', 'A handwritten number 5', 'A handwritten number 6', 'A handwritten number 7', 'A handwritten number 8', 'A handwritten number 9']


In [74]:
import numpy as np 

TOTAL = len(mnist)
CORRECT = 0


for i in range(len(mnist)):
  with torch.no_grad():
    image = preprocess(mnist[i]['image']).unsqueeze(0).to(device)

    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    predict = np.argmax(probs[0])

    if predict == mnist[i]['label']: CORRECT = CORRECT + 1

  if i % 1000 == 0 and i > 0 :
    print(f'Accuracy: {CORRECT}/ {i} ({CORRECT*100.0//i})')

Accuracy: 176/ 1000 (0)
Accuracy: 369/ 2000 (0)
Accuracy: 564/ 3000 (0)
Accuracy: 767/ 4000 (0)
Accuracy: 969/ 5000 (0)
Accuracy: 1165/ 6000 (0)
Accuracy: 1358/ 7000 (0)
Accuracy: 1575/ 8000 (0)
Accuracy: 1788/ 9000 (0)


In [76]:
    print(f'Accuracy: {CORRECT}/ {i} ({CORRECT*100.0//i})')

Accuracy: 2004/ 9999 (20.0)
